In [1]:
import torch
from src import BayesianLinear

In [2]:
linear = BayesianLinear(in_features=3, out_features=5, bias=True)
a = torch.ones(size=(2, 3))

In [3]:
linear(a), linear(a)

(tensor([[ 0.4133, -1.9359,  1.5662,  3.8535, -6.2014],
         [ 1.3407, -6.6935,  2.6942,  2.2106,  2.6592]],
        grad_fn=<ViewBackward0>),
 tensor([[ 3.8239,  5.2796,  4.4794,  2.7578, -0.8376],
         [-5.2359,  5.4935,  0.5253, -1.1673,  2.1211]],
        grad_fn=<ViewBackward0>))

In [4]:
linear.kl

tensor(0., grad_fn=<AddBackward0>)